In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

import functools

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# Загружаем набор чатов на которых будем обучаться
text = open('/content/gdrive/My Drive/Colab Notebooks/myMessages.txt', 'rb').read().decode(encoding='utf-8')
print ('Общее количество символов: {}'.format(len(text)))

# Составляем словарь символов
vocab = sorted(set(text))
print ('Уникальных символов: {}'.format(len(vocab)))

# Функции для преобразования текста в массив чисел и обратно
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

Общее количество символов: 4676255
Уникальных символов: 93


In [0]:
# Подгатавливаем обучающий датасет
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
for input_example, target_example in  dataset.take(1):
  print ('Вход:  ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Выход:', repr(''.join(idx2char[target_example.numpy()])))

Instructions for updating:
Colocations handled automatically by placer.
Вход:   '< йоу\n===\n< уууу\n< неее\n< капец\n< я не про этооо\n< дээ\n< ой\n< не то\n> так ты все таки про это ?\n< ды'
Выход: ' йоу\n===\n< уууу\n< неее\n< капец\n< я не про этооо\n< дээ\n< ой\n< не то\n> так ты все таки про это ?\n< дыа'


In [9]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

SHUFFLE_BUFFER_SIZE = 10000
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Размер используемого словаря
vocab_size = len(vocab)
# Размерности сети
embedding_dim = 256
rnn_units = 1024

# Здесь можно попробовать различные варианты архитектуры сети. На текущий момент лучший вариант у LSTM.

#rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
rnn = tf.keras.layers.CuDNNLSTM
#rnn = tf.keras.layers.CuDNNGRU

# Для ускорения работы на GPU можно использовать rnn = tf.keras.layers.CuDNNGRU, но такая сеть не может быть потом 
# использована для работы на CPU.

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  return tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
      rnn(rnn_units,
          return_sequences=True,
          recurrent_initializer='glorot_uniform',
          stateful=True),
      tf.keras.layers.Dense(vocab_size)
  ])

In [11]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23808     
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (64, None, 1024)          5251072   
_________________________________________________________________
dense (Dense)                (64, None, 93)            95325     
Total params: 5,370,205
Trainable params: 5,370,205
Non-trainable params: 0
_________________________________________________________________


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# Настройка сохранения результатов
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [14]:
history = model.fit(dataset.repeat(), epochs=5, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/5
729/730 [============================>.] - ETA: 0s - loss: 1.9760WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
730/730 [==============================] - 59s 81ms/step - loss: 1.9756
Epoch 2/5
730/730 [==============================] - 58s 80ms/step - loss: 1.4878
Epoch 3/5
730/730 [==============================] - 57s 79ms/step - loss: 1.3851
Epoch 4/5
730/730 [==============================] - 58s 79ms/step - loss: 1.3316
Epoch 5/5
730/730 [==============================] - 58s 79ms/step - loss: 1.2940


In [15]:
# Загружаем модель из checkpoint'а, при этом используем batch_size размера 1, чтобы можно было использовать 
# модель в режиме чата
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23808     
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_1 (Dense)              (1, None, 93)             95325     
Total params: 5,370,205
Trainable params: 5,370,205
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, oneString, temperature):
  # Максимальное количество генерируемых символов
  num_generate = 200
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    if temperature > 0:
        predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    c = idx2char[predicted_id]
    text_generated.append(c)
    if c == '\n' and oneString:
        break

  return (start_string + ''.join(text_generated))

In [35]:
print(generate_text(model, "===", False, 0.4))

====
===
> ураа
< шо так сделал
< вот так скучно
< да
< а ты не сказала
> ну лан
> потом на улице
< неееет
< не могу подождите в туалет
< потом пойдем
< на всё просто пойдем
< как ты могла
> почему ?
< н


In [34]:
dialog = u"===\n"
while(True):
  rq = input("> ")
  if rq == '':
        break;
  dialog += "> " + rq + "\n< "
  
  fullAns = generate_text(model, start_string=dialog, temperature=0.1, oneString = True)
  shortAns = fullAns[len(dialog):]
  print("< " + shortAns)
  dialog = fullAns

> hi
< как дела?

> хорошо, у тебя как?
< нееет

> почему?
< нееет

> алина
< неееет

> алина
< неееет

> нет?
< неееет

> омг
< нееет

> что с тобой?
< нееет

> как дела?
< неееет

> привет
< неееет

> ээээээ
< нееет

> неееет
< не плачь

> неееет
< не плачь

> не плачь
< не плачь

> бля
< не плачь

> тупой
< неееет

> кек
< неееет

> лох
< неееет

> вхахаххахааха
< нееет



KeyboardInterrupt: ignored